In [42]:
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries
import pandas_datareader as web
import pandas as pd
import numpy as np

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [43]:
stockSymbol = "MSFT"
market = "NASDAQ"
invest = 10000
window = 23
start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2017,1,1)
split_test = pd.datetime(2017,10,1)

In [36]:
ts = TimeSeries(key='09WM396BF6PFJRPT', output_format='pandas')
tsData, meta_data = ts.get_daily(symbol=stockSymbol, outputsize='full')
tsData.columns = ['Volume', 'Close', 'High', 'Open', 'Low']

ti = TechIndicators(key='09WM396BF6PFJRPT', output_format='pandas')
data, meta_data = ti.get_bbands(symbol=stockSymbol, interval='daily', time_period=20, series_type='close')
data.index = pd.to_datetime(data.index)
data = data[start_date:split_validation]
tiBbands = data

data, meta_data = ti.get_macd(symbol=stockSymbol, interval='daily', series_type='close')
data.index = pd.to_datetime(data.index)
data = data[start_date:split_validation]
tiMacd = data

tsData.index = pd.to_datetime(tsData.index)
tsData = tsData[start_date:split_validation]

df = pd.concat([tsData, tiBbands, tiMacd, tiRsi], axis=1, join='inner')
df

In [65]:
data, meta_data = ti.get_rsi(symbol=stockSymbol, interval='daily', time_period=20, series_type='close')
data.index = pd.to_datetime(data.index)
data = data[start_date:split_validation]
tiRsi = data

ValueError: Expecting value: line 1 column 1 (char 0)

In [ ]:
tsData.index = pd.to_datetime(tsData.index)
tsData = tsData[start_date:split_validation]

df = pd.concat([tsData, tiBbands, tiMacd, tiRsi], axis=1, join='inner')
df

In [ ]:
#Open window
df['Open-0'] = df.Open
for i in range(1,window):
    df['Open-'+str(i)] = df.Close.shift(i)
    df['Open Change-'+str(i-1)] = df['Open-'+str(i-1)] - df['Open-'+str(i)] 
    df['Open-'+str(i)+' Trend'] = df['Open Change-'+str(i-1)].apply(np.sign)
df.drop('Open-0',axis=1, inplace=True)

In [ ]:
#High window
df['High-0'] = df.High
for i in range(1,window):
    df['High-'+str(i)] = df.Close.shift(i)
    df['High Change-'+str(i-1)] = df['High-'+str(i-1)] - df['High-'+str(i)] 
    df['High-'+str(i)+' Trend'] = df['High Change-'+str(i-1)].apply(np.sign)
df.drop('High-0',axis=1, inplace=True)

In [ ]:
#Low window
df['Low-0'] = df.Low
for i in range(1,window):
    df['Low-'+str(i)] = df.Close.shift(i)
    df['Low Change-'+str(i-1)] = df['Low-'+str(i-1)] - df['Low-'+str(i)] 
    df['Low-'+str(i)+' Trend'] = df['Low Change-'+str(i-1)].apply(np.sign)
df.drop('Low-0',axis=1,  inplace=True)

In [ ]:
#Close window
df['Close-0'] = df.Close
for i in range(1,window):
    df['Close-'+str(i)] = df.Close.shift(i)
    df['Close Change-'+str(i-1)] = df['Close-'+str(i-1)] - df['Close-'+str(i)] 
    df['Close-'+str(i)+' Trend'] = df['Close Change-'+str(i-1)].apply(np.sign)
df.drop('Close-0',axis=1, inplace=True)

In [59]:
#BBand window
df['BBand-Upper-0'] = df['Real Upper Band']
df['BBand-Lower-0'] = df['Real Lower Band']
df['BBand-Middle-0'] = df['Real Middle Band']

for i in range(1,window):
    df['BBand-Upper-'+str(i)] = df.Close.shift(i)
    df['BBand-Upper Change-'+str(i-1)] = df['BBand-Upper-'+str(i-1)] - df['BBand-Upper-'+str(i)] 
    df['BBand-Upper-'+str(i)+' Trend'] = df['BBand-Upper Change-'+str(i-1)].apply(np.sign)
    
    df['BBand-Lower-'+str(i)] = df.Close.shift(i)
    df['BBand-Lower Change-'+str(i-1)] = df['BBand-Lower-'+str(i-1)] - df['BBand-Lower-'+str(i)] 
    df['BBand-Lower-'+str(i)+' Trend'] = df['BBand-Lower Change-'+str(i-1)].apply(np.sign)

    df['BBand-Middle-'+str(i)] = df.Close.shift(i)
    df['BBand-Middle Change-'+str(i-1)] = df['BBand-Middle-'+str(i-1)] - df['BBand-Middle-'+str(i)] 
    df['BBand-Middle-'+str(i)+' Trend'] = df['BBand-Middle Change-'+str(i-1)].apply(np.sign)
    
df.drop('BBand-Upper-0',axis=1, inplace=True)
df.drop('BBand-Lower-0',axis=1, inplace=True)
df.drop('BBand-Middle-0',axis=1, inplace=True)

In [63]:
#Macd window
df['MACD_Hist-0'] = df['MACD_Hist']
df['MACD-0'] = df['MACD']
df['MACD_Signal-0'] = df['MACD_Signal']
for i in range(1,window):
    df['MACD_Hist-'+str(i)] = df.Close.shift(i)
    df['MACD_Hist Change-'+str(i-1)] = df['MACD_Hist-'+str(i-1)] - df['MACD_Hist-'+str(i)] 
    df['MACD_Hist-'+str(i)+' Trend'] = df['MACD_Hist Change-'+str(i-1)].apply(np.sign)

    df['MACD-'+str(i)] = df.Close.shift(i)
    df['MACD Change-'+str(i-1)] = df['MACD-'+str(i-1)] - df['MACD-'+str(i)] 
    df['MACD-'+str(i)+' Trend'] = df['MACD Change-'+str(i-1)].apply(np.sign)

    df['MACD_Signal-'+str(i)] = df.Close.shift(i)
    df['MACD_Signal Change-'+str(i-1)] = df['MACD_Signal-'+str(i-1)] - df['MACD_Signal-'+str(i)] 
    df['MACD_Signal-'+str(i)+' Trend'] = df['MACD_Signal Change-'+str(i-1)].apply(np.sign)
    
df.drop('MACD_Hist-0',axis=1, inplace=True)
df.drop('MACD-0',axis=1, inplace=True)
df.drop('MACD_Signal-0',axis=1, inplace=True)

In [64]:
df

,Volume,Close,High,Open,Low,Real Upper Band,Real Lower Band,Real Middle Band,MACD_Hist,MACD,MACD_Signal,Open-0,Open-1,Open Change-0,Open-1 Trend,Open-2,Open Change-1,Open-2 Trend,Open-3,Open Change-2,Open-3 Trend,Open-4,Open Change-3,Open-4 Trend,Open-5,Open Change-4,Open-5 Trend,Open-6,Open Change-5,Open-6 Trend,Open-7,Open Change-6,Open-7 Trend,Open-8,Open Change-7,Open-8 Trend,Open-9,Open Change-8,Open-9 Trend,Open-10,Open Change-9,Open-10 Trend,Open-11,Open Change-10,Open-11 Trend,Open-12,Open Change-11,Open-12 Trend,Open-13,Open Change-12,Open-13 Trend,Open-14,Open Change-13,Open-14 Trend,Open-15,Open Change-14,Open-15 Trend,Open-16,Open Change-15,Open-16 Trend,Open-17,Open Change-16,Open-17 Trend,Open-18,Open Change-17,Open-18 Trend,Open-19,Open Change-18,Open-19 Trend,Open-20,Open Change-19,Open-20 Trend,Open-21,Open Change-20,Open-21 Trend,Open-22,Open Change-21,Open-22 Trend,High-0,High-1,High Change-0,High-1 Trend,High-2,High Change-1,High-2 Trend,High-3,High Change-2,High-3 Trend,High-4,High Change-3,High-4 Trend,High-5,High Change-4,High-5 Trend,High-6,High Change-5,High-6 Trend,High-7,High Change-6,High-7 Trend,High-8,High Change-7,High-8 Trend,High-9,High Change-8,High-9 Trend,High-10,High Change-9,High-10 Trend,High-11,High Change-10,High-11 Trend,High-12,High Change-11,High-12 Trend,High-13,High Change-12,High-13 Trend,High-14,High Change-13,High-14 Trend,High-15,High Change-14,High-15 Trend,High-16,High Change-15,High-16 Trend,High-17,High Change-16,High-17 Trend,High-18,High Change-17,High-18 Trend,High-19,High Change-18,High-19 Trend,High-20,High Change-19,High-20 Trend,High-21,High Change-20,High-21 Trend,High-22,High Change-21,High-22 Trend,Low-0,Low-1,Low Change-0,Low-1 Trend,Low-2,Low Change-1,Low-2 Trend,Low-3,Low Change-2,Low-3 Trend,Low-4,Low Change-3,Low-4 Trend,Low-5,Low Change-4,Low-5 Trend,Low-6,Low Change-5,Low-6 Trend,Low-7,Low Change-6,Low-7 Trend,Low-8,Low Change-7,Low-8 Trend,Low-9,Low Change-8,Low-9 Trend,Low-10,Low Change-9,Low-10 Trend,Low-11,Low Change-10,Low-11 Trend,Low-12,Low Change-11,Low-12 Trend,Low-13,Low Change-12,Low-13 Trend,Low-14,Low Change-13,Low-14 Trend,Low-15,Low Change-14,Low-15 Trend,Low-16,Low Change-15,Low-16 Trend,Low-17,Low Change-16,Low-17 Trend,Low-18,Low Change-17,Low-18 Trend,Low-19,Low Change-18,Low-19 Trend,Low-20,Low Change-19,Low-20 Trend,Low-21,Low Change-20,Low-21 Trend,Low-22,Low Change-21,Low-22 Trend,Close-0,Close-1,Close Change-0,Close-1 Trend,Close-2,Close Change-1,Close-2 Trend,Close-3,Close Change-2,Close-3 Trend,Close-4,Close Change-3,Close-4 Trend,Close-5,Close Change-4,Close-5 Trend,Close-6,Close Change-5,Close-6 Trend,Close-7,Close Change-6,Close-7 Trend,Close-8,Close Change-7,Close-8 Trend,Close-9,Close Change-8,Close-9 Trend,Close-10,Close Change-9,Close-10 Trend,Close-11,Close Change-10,Close-11 Trend,Close-12,Close Change-11,Close-12 Trend,Close-13,...,BBand-Lower-14 Trend,BBand-Middle-14,BBand-Middle Change-13,BBand-Middle-14 Trend,BBand-Lower-15,BBand-Lower Change-14,BBand-Lower-15 Trend,BBand-Middle-15,BBand-Middle Change-14,BBand-Middle-15 Trend,BBand-Lower-16,BBand-Lower Change-15,BBand-Lower-16 Trend,BBand-Middle-16,BBand-Middle Change-15,BBand-Middle-16 Trend,BBand-Lower-17,BBand-Lower Change-16,BBand-Lower-17 Trend,BBand-Middle-17,BBand-Middle Change-16,BBand-Middle-17 Trend,BBand-Lower-18,BBand-Lower Change-17,BBand-Lower-18 Trend,BBand-Middle-18,BBand-Middle Change-17,BBand-Middle-18 Trend,BBand-Lower-19,BBand-Lower Change-18,BBand-Lower-19 Trend,BBand-Middle-19,BBand-Middle Change-18,BBand-Middle-19 Trend,BBand-Lower-20,BBand-Lower Change-19,BBand-Lower-20 Trend,BBand-Middle-20,BBand-Middle Change-19,BBand-Middle-20 Trend,BBand-Lower-21,BBand-Lower Change-20,BBand-Lower-21 Trend,BBand-Middle-21,BBand-Middle Change-20,BBand-Middle-21 Trend,BBand-Lower-22,BBand-Lower Change-21,BBand-Lower-22 Trend,BBand-Middle-22,BBand-Middle Change-21,BBand-Middle-22 Trend,MACD_Hist-1,MACD_Hist Change-0,MACD_Hist-1 Trend,MACD-1,MACD Change-0,MACD-1 